References:

1. https://github.com/xbresson/CS6208_2023/blob/main/codes/labs_lecture07/01_vanilla_graph_transformers.ipynb
2. https://github.com/pgniewko/pytorch_geometric/blob/master/torch_geometric/nn/conv/transformer_conv.py
3. https://arxiv.org/abs/2012.09699
4. https://arxiv.org/abs/1703.04977

TDC:
1. https://tdcommons.ai/benchmark/overview/

In [1]:
import torch
import math
import torch.nn.functional as F
import numpy as np
from torch import nn
from torch_geometric.data import Data
from torch_geometric.nn import MessagePassing
from torch_scatter.composite import scatter_softmax
from torch_scatter.scatter import scatter_add
from torch_geometric.utils import softmax
from torch_geometric.nn.resolver import activation_resolver
from torch_geometric.nn.aggr import MultiAggregation
from torch_geometric.loader import DataLoader
from torch_geometric.data import Dataset
import os.path as osp
from torch_geometric.datasets import ZINC
from torch_geometric.loader import DataLoader
from torch_geometric.utils import degree
from torch.optim.lr_scheduler import ReduceLROnPlateau

torch.manual_seed(192837465)

# Generate example data
x = torch.randn(6, 3)  # Node features (6 nodes, 3-dimensional features)
edge_index = torch.tensor([[0, 1, 1, 2, 2, 3, 4, 5], [1, 0, 2, 1, 3, 2, 5, 4]], dtype=torch.long)  # Edge indices
edge_attr = torch.randn(8, 2)  # Edge attributes (8 edges, 2-dimensional attributes)

data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)

/Users/pawel/anaconda3/envs/gt/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [47]:
class MLP(nn.Module):
    def __init__(self, dims, output_dim, num_layers=2, dropout=0.0, act='relu', act_kwargs=None):
        super(MLP, self).__init__()
    
        if isinstance(dims, int):
            dims = [dims] * num_layers
            
        assert len(dims) == num_layers
        
        layers = []
        
        for (i_dim, o_dim) in zip(dims[:-1], dims[1:]):
            layers.append(nn.Linear(i_dim, o_dim, bias=True))
            layers.append(activation_resolver(act, **(act_kwargs or {})))
            if dropout > 0:
                layers.append(nn.Dropout(p=dropout))
                
        layers.append(nn.Linear(dims[-1], output_dim, bias=True))
        
        self.mlp = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.mlp(x)

    
class GTConv(MessagePassing):
    def __init__(self, node_in_dim, hidden_dim, edge_in_dim=None, num_heads=1, dropout=0.0, norm='bn', act='relu'):
        super(GTConv, self).__init__(node_dim=0, aggr='add')
        
        assert hidden_dim % num_heads == 0
        assert (edge_in_dim is None) or (edge_in_dim > 0)
        
        self.WQ = nn.Linear(node_in_dim, hidden_dim, bias=True)
        self.WK = nn.Linear(node_in_dim, hidden_dim, bias=True)
        self.WV = nn.Linear(node_in_dim, hidden_dim, bias=True)
        self.WO = nn.Linear(hidden_dim, node_in_dim, bias=True)
        
        if edge_in_dim is not None:
            assert node_in_dim == edge_in_dim
            self.WE = nn.Linear(edge_in_dim, hidden_dim, bias=True)
            self.WOe = nn.Linear(hidden_dim, edge_in_dim, bias=True)
            self.ffn_e = MLP(dims=edge_in_dim, output_dim=edge_in_dim,
                             num_layers=2, dropout=dropout, act=act)
            if norm.lower() in ['bn', 'batchnorm', 'batch_norm']:
                self.norm1e = nn.BatchNorm1d(edge_in_dim)
                self.norm2e = nn.BatchNorm1d(edge_in_dim)
            elif norm.lower() in ['ln', 'layernorm', 'layer_norm']:
                self.norm1e = nn.LayerNorm(edge_in_dim)
                self.norm2e = nn.LayerNorm(edge_in_dim)
        else:
            self.WE = self.register_parameter('WE', None)
            self.WOe = self.register_parameter('WOe', None)
            self.ffn_e = self.register_parameter('ffn_e', None)
            self.norm1e = self.register_parameter('norm1e', None)
            self.norm2e = self.register_parameter('norm2e', None)
        
        if norm.lower() in ['bn', 'batchnorm', 'batch_norm']:
            self.norm1 = nn.BatchNorm1d(node_in_dim)
            self.norm2 = nn.BatchNorm1d(node_in_dim)
        elif norm.lower() in ['ln', 'layernorm', 'layer_norm']:
            self.norm1 = nn.LayerNorm(node_in_dim)
            self.norm2 = nn.LayerNorm(node_in_dim)
            
        self.dropout_layer = nn.Dropout(p=dropout)
            
        self.ffn = MLP(dims=node_in_dim, output_dim=node_in_dim,
                       num_layers=2, dropout=dropout, act=act)
        
        self.num_heads = num_heads
        self.node_in_dim = node_in_dim
        self.edge_in_dim = edge_in_dim
        self.hidden_dim = hidden_dim
        self.dropout = dropout
        self.norm = norm.lower()
        
        self.reset_parameters()
        
        
    def reset_parameters(self):
        # TODO: init linear layers with xavier weights    
        super().reset_parameters()
        
    
    def forward(self, x, edge_index, edge_attr=None):
        #print(edge_index)
        x_ = x
        Q = self.WQ(x).view(-1, self.num_heads, self.hidden_dim // self.num_heads)
        K = self.WK(x).view(-1, self.num_heads, self.hidden_dim // self.num_heads)
        V = self.WV(x).view(-1, self.num_heads, self.hidden_dim // self.num_heads)
        
        out = self.propagate(edge_index, Q=Q, K=K, V=V,
                             edge_attr=edge_attr, size=None)
        out = out.view(-1, self.hidden_dim)
        
        ## NODES
        out = self.dropout_layer(out)
        out = self.WO(out) + x_ # Residual connection
        out = self.norm1(out)
        # FFN-NODES
        ffn_in = out
        out = self.ffn(out)
        out = self.norm2(ffn_in + out)
        
        if self.edge_in_dim is None:
            out_eij = None
        else:
            out_eij = self._eij
            self._eij = None
            out_eij = out_eij.view(-1, self.hidden_dim)

            ## EDGES
            out_eij_ = out_eij
            out_eij = self.dropout_layer(out_eij)
            out_eij = self.WOe(out_eij) + out_eij_ # Residual connection
            out_eij = self.norm1e(out_eij)
            # FFN-EDGES
            ffn_eij_in = out_eij
            out_eij = self.ffn_e(out_eij)
            out_eij = self.norm2e(ffn_eij_in + out_eij)
        
        
        return (out, out_eij)
        
        
    def message(self, Q_i, K_j, V_j, index, edge_attr=None):
        d_k = Q_i.size(-1)
        
        if self.WE is not None:
            assert edge_attr is not None
            E = self.WE(edge_attr).view(-1, self.num_heads, self.hidden_dim // self.num_heads)
            K_j = E * K_j
        
        qijk = (Q_i * K_j).sum(dim=-1) / math.sqrt(d_k)
        self._eij = (Q_i * K_j) / math.sqrt(d_k)
        alpha = softmax(qijk, index) # Log-Sum-Exp trick used. No need for clipping (-5,5)
        
        return alpha.view(-1, self.num_heads, 1) * V_j
    
    def __repr__(self) -> str:
        return (f'{self.__class__.__name__}({self.node_in_dim}, '
                f'{self.hidden_dim}, heads={self.num_heads})')
    
    
class GraphTransformerNet(nn.Module):
    def __init__(self, node_dim_in, edge_dim_in,
                 hidden_dim=128, norm='bn',
                 num_gt_layers=4, num_heads=8,
                 aggregators=['sum'],
                 act='gelu', dropout=0.0):
        super(GraphTransformerNet, self).__init__()
        
        self.node_emb = nn.Embedding(node_dim_in, hidden_dim) #nn.Linear(node_dim_in, hidden_dim)
        self.edge_emb = nn.Embedding(edge_dim_in, hidden_dim) #nn.Linear(edge_dim_in, hidden_dim)
        
        self.gt_layers = nn.ModuleList()
        for _ in range(num_gt_layers): 
            self.gt_layers.append(GTConv(node_in_dim=hidden_dim,
                                         hidden_dim=hidden_dim, edge_in_dim=hidden_dim,
                                         num_heads=num_heads, act=act,
                                         dropout=dropout))
        
        self.global_pool = MultiAggregation(aggregators, mode='cat')
        
        num_aggrs = len(aggregators)
        self.mu_mlp = MLP(dims=[num_aggrs * hidden_dim, hidden_dim, hidden_dim], 
                          output_dim=1, num_layers=3, dropout=0.0, act=act)
        self.std_mlp = MLP(dims=[num_aggrs * hidden_dim, hidden_dim, hidden_dim], 
                           output_dim=1, num_layers=3, dropout=0.0, act=act)
        
        
    def forward(self, x, edge_index, edge_attr, batch):
        #print("pre",x.size())
        x = self.node_emb(x.squeeze())
        #print("post",x.size())
        edge_attr = self.edge_emb(edge_attr)

        for gt_layer in self.gt_layers:
            (x, edge_attr) = gt_layer(x, edge_index, edge_attr=edge_attr)

        x = self.global_pool(x, batch)
        mu = self.mu_mlp(x)
        log_var = self.std_mlp(x)
        
        if self.training:
            std = torch.exp(0.5 * log_var)
            eps = torch.randn_like(std)
            return eps * std + mu
        else:
            return mu
        
    def num_parameters(self):   
        trainable_params = filter(lambda p: p.requires_grad, self.parameters())
        count = sum([p.numel() for p in trainable_params])
        return count

In [48]:
gt = GTConv(node_in_dim=3, hidden_dim=6, num_heads=2, edge_in_dim=2)
gt

AssertionError: 

In [39]:
gt(x, edge_index, edge_attr=edge_attr)

torch.Size([8, 6])
torch.Size([8, 2])


RuntimeError: The size of tensor a (2) must match the size of tensor b (6) at non-singleton dimension 1

In [43]:
gt_net = GraphTransformerNet(node_dim_in=3, edge_dim_in=2)

In [ ]:
loader = DataLoader([data], batch_size=128, shuffle=False)
for data in loader:
    x = data.x
    edge_index = data.edge_index
    edge_attr = data.edge_attr
    pred= gt_net(x, edge_index, edge_attr, data.batch)
    print(pred)

## GET ZINC DATA

In [44]:
path = osp.join('.', 'data', 'ZINC')
train_dataset = ZINC(path, subset=True, split='train')
val_dataset = ZINC(path, subset=True, split='val')
test_dataset = ZINC(path, subset=True, split='test')

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128)
test_loader = DataLoader(test_dataset, batch_size=128)

# Compute the maximum in-degree in the training data.
max_degree = -1
for data in train_dataset:
    d = degree(data.edge_index[1], num_nodes=data.num_nodes, dtype=torch.long)
    max_degree = max(max_degree, int(d.max()))

# Compute the in-degree histogram tensor
deg = torch.zeros(max_degree + 1, dtype=torch.long)
for data in train_dataset:
    d = degree(data.edge_index[1], num_nodes=data.num_nodes, dtype=torch.long)
    deg += torch.bincount(d, minlength=deg.numel())

## TRAIN AND EVAL

In [45]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GraphTransformerNet(node_dim_in=21, edge_dim_in=4, num_gt_layers=4, hidden_dim=128, 
                           dropout=0.1).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20,
                              min_lr=0.00001)


print(model)
print(f"Number of params: {model.num_parameters()//1000}k")
def train(epoch):
    model.train()

    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        #print(data.x.size())
        out = model(data.x, data.edge_index, data.edge_attr, data.batch)
        loss = (out.squeeze() - data.y).abs().mean()
        loss.backward()
        total_loss += loss.item() * data.num_graphs
        optimizer.step()
    return total_loss / len(train_loader.dataset)


@torch.no_grad()
def test(loader):
    model.eval()

    total_error = 0
    for data in loader:
        data = data.to(device)
        out = model(data.x, data.edge_index, data.edge_attr, data.batch)
        total_error += (out.squeeze() - data.y).abs().sum().item()
    return total_error / len(loader.dataset)


for epoch in range(1, 301):
    loss = train(epoch)
    val_mae = test(val_loader)
    test_mae = test(test_loader)
    scheduler.step(val_mae)
    print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}, Val: {val_mae:.4f}, '
          f'Test: {test_mae:.4f}')

GraphTransformerNet(
  (node_emb): Embedding(21, 128)
  (edge_emb): Embedding(4, 128)
  (gt_layers): ModuleList(
    (0): GTConv(128, 128, heads=8)
    (1): GTConv(128, 128, heads=8)
    (2): GTConv(128, 128, heads=8)
    (3): GTConv(128, 128, heads=8)
  )
  (global_pool): MultiAggregation([
    SumAggregation(),
  ], mode=cat)
  (mu_mlp): MLP(
    (mlp): Sequential(
      (0): Linear(in_features=128, out_features=128, bias=True)
      (1): GELU(approximate='none')
      (2): Linear(in_features=128, out_features=128, bias=True)
      (3): GELU(approximate='none')
      (4): Linear(in_features=128, out_features=1, bias=True)
    )
  )
  (std_mlp): MLP(
    (mlp): Sequential(
      (0): Linear(in_features=128, out_features=128, bias=True)
      (1): GELU(approximate='none')
      (2): Linear(in_features=128, out_features=128, bias=True)
      (3): GELU(approximate='none')
      (4): Linear(in_features=128, out_features=1, bias=True)
    )
  )
)
Number of params: 734k
torch.Size([6448, 12

KeyboardInterrupt: 